# Scraping

In [2]:
"""
Created on June 2020
author: Jahnic Beck-Joseph
url: https://github.com/Jahnic/Portfolio/tree/master/RealEstate
"""

import pandas as pd
import numpy as np
from random import randint
import random
import time
import re

# Scraping through Chrome webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
# Starting URLs
centris_url = "https://www.centris.ca/en/properties~for-sale?view=Thumbnail"
duproprio = "https://duproprio.com/en/search/list?search=true&is_for_sale=1&with_builders=1&parent=1&pageNumber=1&sort=-published_at"

In [4]:
class Centris:
    """
    This class represents a chrome webdriver object with access to centris.ca.
    
    Attr:
    self.url - starting url for scraping process
    self.data - pandas.DataFrame object containing scraped data
    self.driver - Chrome webdriver
    self.DRIVER_PATH - path to Chrome webdriver
    self.old_DOM - web elements found in previous DOM 
    """
     
    def __init__(self, url=centris_url): 
        self.url = url
        self.data = pd.DataFrame()
        # Path to Chromedriver
        self.DRIVER_PATH = "/usr/bin/chromedriver"
        self.driver = None
        # Verification of new DOM
        self.old_DOM = {\
                        'title' : [],\
                        'address' : [],\
                        'price' : [],\
                        'lat' : [],\
                        'long' : [],\
                        'descriptions' : [],\
                        'neighbourhood_top' : [],\
                        # 'neighbourhood_mid' : [],\
                        # 'neighbourhood_buttom' : [],\
                        'demographics_buttons' : [],\
                    }
        
    def reset_old_DOM(self):
        self.old_DOM = {\
                        'title' : [],\
                        'address' : [],\
                        'price' : [],\
                        'lat' : [],\
                        'long' : [],\
                        'descriptions' : [],\
                        'neighbourhood_top' : [],\
                        # 'neighbourhood_mid' : [],\
                        # 'neighbourhood_buttom' : [],\
                        'demographics_buttons' : [],\
                    }

    def append_data(self, title, address, price,\
            lat, long, descriptions, neighbourhood_indicators,\
            population, demographics):
        """Appends new data to existing data frame.
        
        Args:
        title - string
        address - string 
        price - 
        lat - 
        long - 
        descriptions - 
        neighbourhood_indicators -
        population - 
        demographics - 
        """
        new_data = pd.DataFrame({\
                        'title': title,\
                        'address': address,\
                        'price': price,\
                        'lat': lat,\
                        'long': long\
                    }, index=[0])

        # DESCRIPTIONS
        description_table = pd.DataFrame()
        for key in descriptions.keys():
            header = key
            value = descriptions[header]
            description_table[header] = pd.Series(value)
        
        # POPULATION AND DEMOGRAPHICS
        new_data = pd.concat([new_data, neighbourhood_indicators, description_table,\
                             population, demographics], axis=1)

        # URL
        new_data['url'] = self.url

        # LOGGING --------------------------     
        # print(new_data)
        self.data = self.data.append(new_data, sort=False,\
                                     ignore_index=True)
        
    
    def get_data(self):
        return self.data
        
    def start_driver(self):
        """
        Starts and returns Crome webdriver. 
        The page link in the url attribute 
        is opened in headless mode.
        """
        
        # Activate headless mode for fastest response
        options = Options()
        options.add_argument('--start-maximized') # open Browser in maximized mode
        options.add_argument('--incognito')
        options.add_argument("--headless")
        options.add_argument("--disable-dev-shm-usage"); # overcome limited resource problems
        
        # Start driver with url
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.set_window_size(1120, 1000)
        self.driver.get(self.url)

    def sort_listings(self):
        """Sorts listings in webdriver from newest to oldest."""
        
        # Click drop down menu
        drop_down = self.driver.find_element_by_xpath(\
                                    "//button[@id='dropdownSort']")
        drop_down.click()
        
        # Sort by most recent listings
        sort_by = self.driver.find_element_by_xpath('//a[@data-option-value="3"]')
        sort_by.click()
    
    def goto_first_page(self):
        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='goFirst']")
            next_page.click()
        except:
            print("goFirst button not available")
    
    def next_page(self):
        # Return to top of page, to access next-page button
        body = self.driver.find_element_by_tag_name("body")
        body.send_keys(Keys.HOME)
        for i in range(2):
            body.send_keys(Keys.PAGE_UP)

        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='next']")
            next_page.click()
        except:
            time.sleep(0.5)
            # Try again after waiting 0.5 sec.
            try:
                next_page = self.driver.find_element_by_xpath(\
                                            "//li[@class='next']")
                next_page.click()
            except:
                print("Next-page button not found!")
                
    def get_page_position(self):
        '''Returns the first and last page of the current search.
        
        Returns
        tuple - (current_page, last_page), '''
        
        pages = self.driver.find_element_by_xpath(\
                                    "//li[@class='pager-current']").text.\
                                    split(" / ")
        
        current_page, last_page = (int(page.replace(",","")) for page in pages)
        
        return (current_page, last_page)
    
    def refresh_page(self):
        "Refreshes current webdriver page."
        self.driver.refresh()
        print("Page is being refreshed.")
        # Wait until page fully loaded
        time.sleep(3)
        
    def distance(origin, destination):
        """Calculates distances from latitudinal/longitudinal data using
        the haversine formula"""
        lat1, lon1 = origin
        lat2, lon2 = destination
        radius = 6371 # km
        
        #Convert from degrees to radians
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        
        # Haversine formula
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c

        return d
                
                                                 
# Instantiate class object
centris = Centris()

The following functions need to be outside of the Class. wait_for_xpath() determined the approptiate time to call get_data(). Initially, both fuctions were part of the class object. It seems that after the get_data() call, the driver does not get updated within the class. This leads in some cases to old DOM's being accessed after the browser has already switched to the next page. To circumvent this issue, elements are called outside the class and tried until accessible. This allows the entire new DOM to be loaded before get_data() is called.

In [5]:
 def wait_for_xpath(xpath: str, old_element):
        """
        Wait until elements in new DOM are accessible.
        
        Arg.
        xpath - xpath to new element 
        old_element - element at xpath from previous DOM (found in centris.old_DOM)
        
        Returns:
        current_element - the element found in the new DOM at xpath
        """
        
        centris_driver = centris.driver
        element_at_xpath = []
        
        # Ensure that the NEW rather than the previous or no DOM is active
        # Maximum wait time 10 sec.
        time_passed = 0
        
        # # ---------------------- LOGGING
        # print('Waiting for xpath')
        
        while (\
            (element_at_xpath == old_element or  element_at_xpath == [])\
            and (time_passed <= 10)\
              ):
            # # ------------------------- LOGGING
            # print("Old element:", element_at_xpath == old_element)
            # print("Empty element:", element_at_xpath == [])
            
            # Wait for DOM to load
            time.sleep(0.2)
            time_passed += 0.2
            
            # # Print every 2 seconds
            # if time_passed%2 == 0:
            #     print("Waiting for new DOM...")
            
            # Attempt to load new DOM
            try: 
                element_at_xpath = centris_driver.find_elements_by_xpath(xpath)
            except: pass
        
        # time-out restart 
        if time_passed > 10:
            print("RuntimeError: element not found.")
            centris.driver.refresh()
            time.sleep(5)
            get_data_from_centris()
            wait_for_xpath(xpath, old_element)
            
        return element_at_xpath

In [6]:
def scrape_description(old_DOM):
    """ Requires instantiated centris object. Scrapes and returns
    description data: Year build, price, Net area, etc."""
    
    descriptions = wait_for_xpath("//div[@class='col-lg-12 description']",\
                                 old_DOM)
    #First three elements not relevant
    descriptions_list = descriptions[0].text.split("\n")[3:]
    
    #LOGGING------------------------
    #print("DESCRIPTION:", descriptions_list)
    
    # Update old_DOM dictionary with new element for next verification
    centris.old_DOM['descriptions'] = descriptions
    
    return extract_descriptions(descriptions_list)

In [7]:
# The data_dict found on this part of the page is inconsistent across listings
    # The first row may contain the number of rooms, bedrooms and bathrooms without headers or may be missing
    # Following rows have heathers with associated values after a line break
    # The very last element may be a walking score without header
    # Listings without first row may supply first row information in subsequent rows with headers
    # Because of these inconsistencies, two seperate extractions need to be implemented: one for
    # first row lements (if they exist) and another for subsequent rows

def extract_descriptions(descriptions_list):
    """Takes in data from scrape_description() and returns it 
    as a dictionary"""
    
    # Transformed data
    data_dict = {}
    # Distinguish between elements from first and subsequent rows if first row exists
    first_row = True
    # Starting point for second part of transformation
    second_row_index = 0
    
    # First Part
    while first_row == True:
        for description in descriptions_list:
            numeric = re.findall("\(*[0-9]+\)*", description) # numbers
            text = re.findall("[A-Za-z]+[A-Za-z\s\-]*", description) # text after/inbetween numbers 

            # Initial elements with numeric values correspond to first row
            if (numeric != []):
                # For each value there must be one text description
                if (len(numeric) == len(text)):
                    for description,value in zip(text, numeric):
                        # Save as column in data_dict
                        description_clean = description.replace("and", "").strip()
                        data_dict[description_clean] = value
                    second_row_index += 1 
                else:
                    print("Unequal number of first row keys and values!")
                    print("Numbers:", numeric)
                    print("Text:", text)
                    break
            else:
                first_row = False # No numeric information implies header
                break
    
    # Index range of second extraction
    # Headers are found at every second index (0,2,4,...)
    # Values are one index apart from their corresponding header (1,3,5,...)
    list_length = len(descriptions_list)
    if (list_length - second_row_index)%2 == 1: # Implies presence of element without header -> Walk Score
        walk_score_listed = True
        end_point = list_length -1
    else:
        walk_score_listed = False
        end_point = list_length
    # Indices corresponding to headers
    extraction_range = range(second_row_index, end_point, 2)
    
    #LOGGING----------------------
#     print("Second row index:", second_row_index)
#     print("Extraction range:", extraction_range)
#     print("List:", descriptions_list)
    
    # Second Part
    for header_index in extraction_range:
        # Headers as column names
        header = descriptions_list[header_index]
        # Values corresponding to headers are found at subsequent indices
        information = descriptions_list[header_index + 1] 
        data_dict[header] = information
    
    if walk_score_listed:
        data_dict["walk_score"] = descriptions_list[-1]
        #LOGGING----------------------
        #print("Walk Score:", descriptions_list[-1])
        
    #LOGGING--------------------------
#     print("Descriptions:", data_dict)
        
    return data_dict

In [8]:
# def scrape_neighbourhood(old_DOM_top, old_DOM_mid, old_DOM_buttom):
#     """ Scrapes and returns a list of ratings 
#     between 0-10 for a set of neighborhood indicators
#     such as groceries, parks, noise, etc.)
#     """
#     driver = centris.driver
    
#     # Extract elements from top section of scrollable list
#     neighbourhood_top = wait_for_xpath(\
#                             "//div[@class='ll-list ps ps--active-y']",\
#                             old_DOM_top)
#     # Split into indicators and ranking values
#     top = [x.text for x in neighbourhood_top][0].split("\n")
    
#     # LOGGING----------------------
#     print("Top neighbourhood:", top)
    
#     # Extract middle section - only one element
#     # Scroll and activate scrollable bar container
#     scrollable_bar = driver.find_element_by_xpath(\
#                                             "//div[@class='ps__thumb-y']")
#     ActionChains(driver).\
#         move_to_element(scrollable_bar).\
#         send_keys(Keys.PAGE_DOWN).\
#         click(scrollable_bar).perform()

#     # Elements from buttom of scrollable list
#     neighbourhood_mid = wait_for_xpath(\
#                             "//div[@class='ll-list ps ps--active-y']",\
#                             old_DOM_mid)
#     # Split into indicators and ranking values
    
#     # LOGGING----------------------
#     print("Neighbourhoud mid section:", neighbourhood_mid)
    
#     middle = [x.text for x in neighbourhood_mid][0].split("\n")
    
#     # Extract buttom section
#     # Scroll and load remaining elements
# #     scrollable_bar = driver.find_element_by_xpath(\
# #                                             "//div[@class='ps__thumb-y']")
#     ActionChains(driver).\
#         move_to_element(scrollable_bar).\
#         send_keys(Keys.PAGE_DOWN).\
#         click(scrollable_bar).perform()
    
#     # Elements from buttom of scrollable list
#     neighbourhood_buttom = wait_for_xpath(\
#                             "//div[@class='ll-list ps ps--active-y']",\
#                             old_DOM_buttom)
#     # Split into indicators and ranking values
#     buttom = [x.text for x in neighbourhood_buttom][0].split("\n")
    
#     # LOGGING----------------------
#     print("Buttom neighbourhood:", buttom)
    
#     # Unite all three sections by storing tuples of indicator names and corresponding values
#     united_list = []
#     list_length = len(top)
#     for i in range(0, list_length, 2):
#         united_list.append((top[i], top[i+1]))
#         united_list.append((middle[i], middle[i+1]))
#         united_list.append((buttom[i], buttom[i+1]))
    
#     # Create set of unique tuples
#     neighbourhood_indicators = set(united_list)
    
#     # LOGGING----------------------
#     print("Number of neighborhood indicators: ", len(neighbourhood_indicators))
#     print("UNITED:", united_list)
#     print("SET:", neighbourhood_indicators)
    
#     # Verify size and extract information as list
#     # If size unexpected, refresh page and restart process
#     if len(neighbourhood_indicators) < 8:
#             centris.refresh_page()
#             scrape_neighbourhood(old_DOM_top, old_DOM_buttom)

#     # Update old_DOM dictionary with new elements for next verification
#     centris.old_DOM['neighbourhood_top'] = neighbourhood_top
#     centris.old_DOM['neighbourhood_mid'] = neighbourhood_mid
#     centris.old_DOM['neighbourhood_buttom'] = neighbourhood_buttom

#     return extract_neighbourhood_indicators(neighbourhood_indicators)

In [9]:
def scrape_neighbourhood(old_DOM_top):
    """ Scrapes and returns a list of ratings 
    between 0-10 for a set of neighborhood indicators
    such as groceries, parks, noise, etc.)
    """
    driver = centris.driver
    
    # Extract elements from top section of scrollable list
    neighbourhood_top = wait_for_xpath(\
                            '//ul[@class="ll-module__list"]',\
                            old_DOM_top)
   
    # Split into indicators and ranking values
    top = [x.text for x in neighbourhood_top][0].split("\n")
    
    # Create set of unique tuples
    neighbourhood_indicators = list(top)
    
#     # LOGGING----------------------
#     print("Number of neighborhood indicators: ", len(neighbourhood_indicators))
#     print(len(neighbourhood_top))
#     print("List:", neighbourhood_indicators)
    
    # Verify size and extract information as list
    # If size unexpected, refresh page and restart process
    if len(neighbourhood_indicators) < 30:
            print("Incorrect number of neighbourhood indicators")
            centris.driver.refresh()
            time.sleep(3)
            scrape_neighbourhood(old_DOM_top)

    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['neighbourhood_top'] = neighbourhood_top
    # return indicators
    return extract_neighbourhood_indicators(neighbourhood_indicators)

In [10]:
def extract_neighbourhood_indicators(indicators):
    """Takes in neighbourhood data from scrape_neighbourhood() and returns it 
    in tabular form as a DataFrame object"""
    data = pd.DataFrame()
    n_indicators = len(indicators)
    for index in range(0, (n_indicators - 1), 2):
        header = indicators[index]
        value = indicators[index + 1]
        data[header] = pd.Series(value)
    
    return data

In [11]:
def scrape_population():
    """Scrapes and returns population summary data (density, variation etc.)"""
    population_summaries =  centris.driver.find_element_by_id('info')
    population_summaries_list = population_summaries\
                        .text.split("\n")
    
    # LOGGING-----------------------
    #print("Population:", population_summaries_list)
    
    return extract_population(population_summaries_list)

In [12]:
def extract_population(population):
    """Takes in population data from scrape_population() and returns it 
    in tabular form as a DataFrame object"""
    
    data = pd.DataFrame()
    for info in population:
        units_removed =  info.replace("hab/km2", "").strip()
        # Numeric data
        numeric = re.findall("[0-9]+[0-9,]*", units_removed)
        numeric_clean = numeric[-1].replace(",","")

        # Text data for column names
        header = re.findall("[a-zA-Z\s]+", units_removed)
        header_clean = header[0]
        # Add numeric data to header excluding the value at index -1
        for numeric_head_data in numeric[:-1]:
            header_clean = header_clean + str(numeric_head_data) + " "

        data[header_clean] = pd.Series(numeric_clean).astype("int")
    return data

In [13]:
def scrape_demographics(old_DOM):
    """Scrapes and return demographic data found in a clickable list"""
    
    driver = centris.driver
    # Clickable list containing demographic data
    menu = driver.find_element_by_id("menu")
    # Load menu by moving browser to it
    ActionChains(driver).\
    move_to_element(menu).perform()
    
    #Buttons to access demographics data (education, incomes, etc.)
    demographics_buttons = wait_for_xpath(\
                        "//div[@class='centrisSocioDemobutton']",\
                                                 old_DOM)

    # LOGGING------------------------
    # print("DEMO. BUTTONS:", demographics_buttons)

    # First entry on clickable demographics list (pre-selected)
    demographics = []

    # Click buttons to access next demogrpahics elements
    for button in demographics_buttons:
        try: 
            button.click()
        except: 
            print("Demographics button missing!")
            # Reattempt loading buttons
            centris.refresh_page()
            ActionChains(driver).\
            move_to_element(menu).\
            perform()
            time.sleep(2) # extra time to load
            demographics_buttons = wait_for_xpath(\
                        "//div[@class='centrisSocioDemobutton']",\
                                                 old_DOM)
            
        # Get and append data after button click
        demographic_data = driver.find_element_by_class_name(\
                         "socioDemoLabel")
        demographics.append(demographic_data.text)
    
    # Split each demographic component into separate list
    # Example: splits "Occupation" data into -> ["Owners", "35%", "Renters", "65%"]
    demographics = [demo.split("\n") for demo in demographics]
    
    #LOGGING------------------------
#     print("DEMO. DATA:", demographics)
#     print("-"*50)
    
    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['demographics_buttons'] = demographics_buttons
    return extract_demographics(demographics)

In [1]:
def extract_demographics(demographics):
    """Takes in demographic data from extract_demographics() and returns it 
    in tabular form as a DataFrame object"""
    
    data = pd.DataFrame()
    for demographic in demographics:
        # Remove empty stings from splitting double line breaks \n\n
        removed_empty_strings = [x for x in demographic if x != ""]
        demographic = removed_empty_strings
        # Format of demographic: [header, value, header, value, ...]
        header_index = range(0, len(demographic), 2)
        for i in header_index:
            header = demographic[i] + " (%)" # add units to column names
            value = demographic[i+1].replace("%", "") # remove units from values 
            data[header] = pd.Series(value).astype("int")

    return data

In [14]:
def get_data_from_centris():
        """
        Requires instantiate Centris object. Scrapes information from the
        webdriver and appends it to the Centris object.
        """
        driver = centris.driver
        old_DOM = centris.old_DOM
        
        # Data from headers
        print("Start scraping new page...")
        title = wait_for_xpath("//span[@data-id='PageTitle']", old_DOM['title'])
        address = wait_for_xpath("//h2[@itemprop='address']", old_DOM['address'])
        price = wait_for_xpath("//span[@itemprop='price']", old_DOM['price'])
        lat = wait_for_xpath("//meta[@itemprop='latitude']", old_DOM['lat'])
        lon = wait_for_xpath("//meta[@itemprop='longitude']", old_DOM['long'])

        # Save new elements as old DOM
        centris.old_DOM['price'] = price
        centris.old_DOM['title'] = title
        centris.old_DOM['address'] = address
        centris.old_DOM['lat'] = lat
        centris.old_DOM['long'] = lon

        # Extract text
        title = title[0].text
        address = address[0].text
        price = price[0].text
        lat = lat[0].get_attribute("content")
        lon = lon[0].get_attribute("content")
        
        # Scrape remaining elements and store in dataframe
        # print('Start TESTING')
        descriptions = scrape_description(old_DOM['descriptions'])
        # print('Descriptions:', descriptions)
        # print('++++++++++++++++++++Neighborhood+++++++++++++++++++++++')
        neighbourhood_indicators = scrape_neighbourhood(old_DOM['neighbourhood_top']
                                                        # old_DOM['neighbourhood_mid'],\
                                                        # old_DOM['neighbourhood_buttom']
                                                            )
        # print('++++++++++++++++++++Demographics+++++++++++++++++++++++')
        population = scrape_population()
        demographics = scrape_demographics(old_DOM['demographics_buttons'])
                
        # Unify data in single dataframe and append to results table
        centris.append_data(
            title,\
            address,\
            price,\
            lat,\
            lon,\
            descriptions,\
            neighbourhood_indicators,\
            population,\
            demographics\
        )
        
        # # LOGGING--------------------------
        # print("GET DATA: DESCRIPTIONS:", descriptions)

## Initiate scraping

In [15]:
# Create driver object
centris = Centris()
start = time.time()
centris.start_driver()
print("Execution time:", time.time() - start)

Execution time: 4.971621036529541


Before running the next cell, search for the region(s) you want to scrape in the webdriver window.
This is not required but will narrow results and reduce runtime.

In [16]:
# Do not include already scraped listings
# centris.sort_listings() # newest listings first
already_scraped = pd.read_csv('data/centris_montreal_condos.csv')
already_scraped = set(already_scraped.address)

In [28]:
start = time.time() 
current_page, last_page = centris.get_page_position() 
pages_to_scrape = last_page - current_page + 1 # in case scraping is interupted
one_to_100 = range(1,100) # to print message each 1% completion
exclude_old_data = True # set False if already scraped listings should be skipped 

print("Scraping initiated.")
print("Total number of pages to scrape:", pages_to_scrape)
print("Estimated runtime:", round(pages_to_scrape*((10)/(60*60)), 2), "hours")
print("="*50)

for i in range(pages_to_scrape):
    
    print("="*50)
    print("Page:", i+1)
    time_passed = 0 # to exit while loop after 10 seconds

    # Check if listing was already scraped
    if exclude_old_data:
        current_address = wait_for_xpath("//h2[@itemprop='address']", centris.old_DOM['address'])[0]           
        if current_address.text not in already_scraped:          
            get_data_from_centris() # Retrieve data only if not already scraped
        else:
            print(current_address.text)
            print(current_address, 'was already scraped!')
    else:
        get_data_from_centris() # Retrieve data

    
    # Next page click
    centris.next_page()
    time.sleep(1) # wait for next page
    centris.driver.refresh() # refresh to free up RAM
    time.sleep(1)
    
    # Percent completed of scraping 
    percent_complete = round(100*((i)/last_page),2)      
    # Print after every 1% mark
    if percent_complete in one_to_100:
        # Calculate time per page
        execution_time = (time.time() - start)/(i+1) # seconds per page
        print(percent_complete, "%", "completed")
        print("Average execution time per page:", round(execution_time, 2), "sec.")
        print("Estimated remaining runtime:", round(\
                                (last_page - (i+1))\
                                *(execution_time\
                                /(60*60)), 1\
                                ), "hours <", "-"*50)
        print("="*50)


print("Total runtime:", execution_time/(60*60), "hours")
centris.data

Scraping initiated.
Total number of pages to scrape: 1
Estimated runtime: 0.0 hours
Page: 1
Start scraping new page...
Next-page button not found!
Total runtime: 0.0035031242355888273 hours


,title,address,price,lat,long,Pedestrian friendly,Parks,Grocery stores,Vibrancy,Nightlife,...,bedrooms in basement,bedroom in basement,Building style,Lot area,Parking (excluded from the price),< 15 years (%),15 to 24 years (%),25 to 44 years (%),45 to 64 years (%),> 65 years (%)
0,Condo for sale,"90, Rue des Soeurs-Grises, apt. 301, Montréal ...","$559,000",45.4987890000,-73.5545970000,10,10,10,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Condo for sale,"101, Rue de la Rotonde, apt. 702, Montréal (Ve...","$415,000",45.4743275300,-73.5402237500,5,8,6,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Condo for sale,"1982, Rue Beaubien Est, Montréal (Rosemont/La ...","$398,000",45.5445538300,-73.5948958000,10,9,10,9,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Condo for sale,"3521A, Rue University, Montréal (Le Plateau-Mo...","$979,000",45.5068160000,-73.5768740000,10,10,10,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Condo for sale,"5549, Chemin Queen-Mary, apt. 8, Montréal (Côt...","$410,000",45.4807470000,-73.6320300000,10,10,10,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5225,Condo for sale,"4850, Chemin de la Côte-Saint-Luc, apt. PH2, M...","$1,450,000",45.4835190000,-73.6164990000,8,10,9,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5226,Condo for sale,"2301, Rue Saint-Patrick, apt. B509, Montréal (...","$599,000",45.4816740000,-73.5690460000,10,10,9,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5227,Condo for sale,"2301, Rue Saint-Patrick, apt. B215, Montréal (...","$1,050,000",45.4816740000,-73.5690460000,10,10,9,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5228,Condo for sale,"2301, Rue Saint-Patrick, apt. B215, Montréal (...","$1,050,000",45.4816740000,-73.5690460000,10,10,9,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Save new data in centris_montreal_condos.csv
new_data = centris.data.copy()
new_data = new_data[new_data.address.duplicated() == False]
old_data = pd.read_csv('data/centris_montreal_condos.csv')
merged_data = pd.concat([new_data, old_data], ignore_index=True)
merged_data.to_csv('data/centris_montreal_condos.csv')

In [35]:
# Display saved data
pd.read_csv('data/centris_montreal_condos.csv')

,Unnamed: 0,title,address,price,lat,long,Pedestrian friendly,Parks,Grocery stores,Vibrancy,...,Parking (excluded from the price),< 15 years (%),15 to 24 years (%),25 to 44 years (%),45 to 64 years (%),> 65 years (%),Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1
0,0,Condo for sale,"90, Rue des Soeurs-Grises, apt. 301, Montréal ...","$559,000",45.498789,-73.554597,10.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Condo for sale,"101, Rue de la Rotonde, apt. 702, Montréal (Ve...","$415,000",45.474328,-73.540224,5.0,8.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Condo for sale,"1982, Rue Beaubien Est, Montréal (Rosemont/La ...","$398,000",45.544554,-73.594896,10.0,9.0,10.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Condo for sale,"3521A, Rue University, Montréal (Le Plateau-Mo...","$979,000",45.506816,-73.576874,10.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Condo for sale,"5549, Chemin Queen-Mary, apt. 8, Montréal (Côt...","$410,000",45.480747,-73.632030,10.0,10.0,10.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9122,9122,Condo for sale,"1801, Rue Viola-Desmond, apt. 125, Montréal (L...","$319,000",45.450187,-73.606141,7.0,9.0,9.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6158.0,3804.0,1450.0,2106.0
9123,9123,Condo for sale,"150, Rue Sherbrooke Est, apt. 217, Montréal (V...","$649,900",45.513985,-73.568120,10.0,10.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6159.0,3805.0,1451.0,2133.0
9124,9124,Condo for sale,"288, Rue Richmond, apt. 257, Montréal (Le Sud-...","$286,000",45.487839,-73.565664,10.0,10.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6160.0,3806.0,1452.0,2155.0
9125,9125,Condominium house for sale,"15641, Rue Victoria, Montréal (Rivière-des-Pra...","$249,900",45.690711,-73.492837,3.0,8.0,6.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6161.0,3807.0,1453.0,2222.0
